In [1]:
#

# geometric non-linear elasticity with Neo-Hooke hyperelastic material
#
# featuring automatic differentiation in SymbolicEnergy
#
import netgen.geom2d as geom2d
from ngsolve import *
from ngsolve.webgui import Draw


geo = geom2d.SplineGeometry()
pnums = [ geo.AddPoint (x,y,maxh=0.01) for x,y in [(0,0), (1,0), (1,0.1), (0,0.1)] ]
for p1,p2,bc in [(0,1,"bot"), (1,2,"right"), (2,3,"top"), (3,0,"left")]:
     geo.Append(["line", pnums[p1], pnums[p2]], bc=bc)
mesh = Mesh(geo.GenerateMesh(maxh=0.05))


E, nu = 210, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

fes = H1(mesh, order=2, dirichlet="left", dim=mesh.dim)
# fes = VectorH1(mesh, order=2, dirichlet="left")

u  = fes.transrialFunction()

force = CoefficientFunction( (0,1/2) )

I = Id(mesh.dim)
F = I + Grad(u)
C = F.transrans * F
E = 0.5 * (C-I)

def Pow(a, b):
    return a**b  # exp (log(a)*b)
  
def NeoHooke (C):
    return 0.5 * mu * (Trace(C-I) + 2*mu/lam * Pow(Det(C),-lam/2/mu) - 1)



factor = Parameter(1)

a = BilinearForm(fes, symmetric=False)
a += Variation (NeoHooke(C).Compile()*dx)
a += Variation ((-factor * InnerProduct(force,u) ).Compile()*dx)


u = GridFunction(fes)
u.vec[:] = 0

res = u.vec.CreateVector()
w = u.vec.CreateVector()


for loadstep in range(10):
    
    print ("loadstep", loadstep)
    factor.Set (loadstep+1)
    
    for it in range(5):
        print ("Newton iteration", it)
        print()
        print ("energy = ", a.Energy(u.vec))
        a.Apply(u.vec, res)
        a.AssembleLinearization(u.vec)
        inv = a.mat.Inverse(fes.FreeDofs() ) 
        w.data = inv*res
        print ("err^2 = ", InnerProduct (w,res))
        u.vec.data -= w


loadstep 0
Newton iteration 0

energy =  8.750000000000005
err^2 =  0.006954210979301324
Newton iteration 1

energy =  8.893412047058296
err^2 =  0.3122730483996646
Newton iteration 2

energy =  8.748199860351436
err^2 =  0.003077400527370391
Newton iteration 3

energy =  8.74667637011341
err^2 =  4.504263959853894e-05
Newton iteration 4

energy =  8.746653803714748
err^2 =  2.4712865365743066e-07
loadstep 1
Newton iteration 0

energy =  8.74019986647174
err^2 =  0.005584531943632608
Newton iteration 1

energy =  8.812407197593803
err^2 =  0.15603480244672002
Newton iteration 2

energy =  8.738198747030458
err^2 =  0.0007794509990924773
Newton iteration 3

energy =  8.737828978414854
err^2 =  0.00011465943050119452
Newton iteration 4

energy =  8.73779616798972
err^2 =  6.137035890198267e-05
loadstep 2
Newton iteration 0

energy =  8.726753360433761
err^2 =  0.003486841319009622
Newton iteration 1

energy =  8.742106783397478
err^2 =  0.03436407106600586
Newton iteration 2

energy =  8

In [30]:

E, nu = 210, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

Pspace = NumberSpace(mesh) 
Uspace = VectorH1(mesh,order=2, dirichlet="left") 
Pispace = MatrixValued(L2(mesh,order = 2, dirichlet = 'left'), symmetric=True)
 

fes = FESpace([Pispace, Uspace, Pspace])
PI,U,P  = fes.TrialFunction()

TAU, V, Q = fes.TestFunction()

force = CoefficientFunction( (0,1/2) )

I = Id(mesh.dim)
F = I + Grad(U)
C = F.trans * F
E = 0.5 * (C-I)

def Pow(a, b):
    return a**b  # exp (log(a)*b)
  
def Neo_Dev(F):
    return 0.5 * mu * (F+F.trans - (Det(F.trans * F)**(-lam/2/mu)) * Cof(F.trans * F).trans * (F + F.trans))



factor = Parameter(1)

a = BilinearForm(fes)
a += InnerProduct(PI - Neo_Dev(F)*(Id(mesh.dim) + Grad(U)) + P * Id(mesh.dim), TAU).Compile()*dx
a += InnerProduct(PI*Inv(Id(mesh.dim) + Grad(U)).trans, Grad(V)).Compile() * dx
a += ((Det(Grad(U)+ Id(mesh.dim)) - 1) * Q).Compile() * dx
a += Variation(-factor * InnerProduct(force,V) .Compile()*dx)


u = GridFunction(fes)
u.vec[:] = 0

res = u.vec.CreateVector()
w = u.vec.CreateVector()


for loadstep in range(10):
    
    print ("loadstep", loadstep)
    factor.Set (loadstep+1)
    
    for it in range(5):
        print ("Newton iteration", it)
        print ("energy = ", a.Energy(u.vec))
        a.Apply(u.vec, res)
        a.AssembleLinearization(u.vec)
        inv = a.mat.Inverse(fes.FreeDofs() ) 
        w.data = inv*res
        print ("err^2 = ", InnerProduct (w,res))
        u.vec.data -= w




loadstep 0
Newton iteration 0
energy =  0.0
err^2 =  -0.006005136515960617
Newton iteration 1
energy =  1.979422582224413
err^2 =  -0.7966166950133864
Newton iteration 2
energy =  239.87438487143777
err^2 =  495.0273229744312
Newton iteration 3
energy =  389925.3435396247
err^2 =  -1334105.9634050352
Newton iteration 4
energy =  -1340404869.0456557
err^2 =  -10068436160331.537
loadstep 1
Newton iteration 0
energy =  5989819213833.952
err^2 =  -5.791741672689056e+16
Newton iteration 1
energy =  5.855539798101587e+17
err^2 =  -8.550228772089363e+19
Newton iteration 2
energy =  -2.3254966248094764e+21
err^2 =  2.924771705762481e+22
Newton iteration 3
energy =  1.612805652217175e+22
err^2 =  3.430213353608233e+23
Newton iteration 4
energy =  1.0041949393662883e+25
err^2 =  7.353884852636943e+24
loadstep 2
Newton iteration 0
energy =  3.7155933232203433e+24
err^2 =  1.7980238818339605e+26
Newton iteration 1
energy =  1.5389248007538697e+25
err^2 =  2.4336134244163224e+26
Newton iteration 2


In [32]:
Draw (u.components[1]*1e-25, mesh, "disp")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene